# Enviroment notes 
 - Running on python 3.11.7
 - Pandas version 2.2.1

In [1]:
# Data analysis
import pandas as pd
# Data visualization
import matplotlib.pyplot as plt
# Data manipulation
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# load the CSV data into a DataFrame
df = pd.read_csv('Phishing_Email.csv')

In [3]:
# Look at header data
df.head()

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [4]:
# look at features 
df.columns

Index(['Unnamed: 0', 'Email Text', 'Email Type'], dtype='object')

In [5]:
# count of safe emails in the dataset and count of phishing emails in the dataset
df['Email Type'].value_counts()

Email Type
Safe Email        11322
Phishing Email     7328
Name: count, dtype: int64